In [4]:
import os
import pandas as pd
import numpy as np
import joblib
from tqdm import tqdm_notebook

In [10]:
# data_ws = '/cfs/cfs-kur7csnz/jiaokeshi/quant'
data_ws = 'D:\\git_dir\\bojin_campaign\\hklh_quote_api_py\\demo\\history_quote'
data_file = os.path.join(data_ws, 'csi500_20220901_20230901.zip')

In [11]:
df = pd.read_csv(data_file)
df.head()

,code,date,times,open_price,high_price,low_price,close_price,vwap,money,volume
0,000009.SZ,20220902,93000000,14.60,14.60,14.60,14.60,14.600000,49669200000,340200
1,000009.SZ,20220902,93300000,14.60,14.60,14.58,14.58,14.591919,46460670000,318400
2,000009.SZ,20220902,93400000,14.58,14.58,14.56,14.56,14.569582,28570950000,196100
3,000009.SZ,20220902,93500000,14.57,14.57,14.50,14.51,14.524857,34423910000,237000
4,000009.SZ,20220902,93800000,14.56,14.56,14.48,14.48,14.522299,39602310000,272700


In [ ]:
# data_ws = './'
# data_file = './data_sample_code10.csv'
# df = pd.read_csv(data_file)

In [24]:
df_20230901 = df[df['date'] == 20230901]
len(df_20230901)
df_20230901.to_csv('df_20230901.csv', index=False)

In [31]:
df_20230901['times'].max()

150000000

In [38]:
"{:0>6d}".format(int(93800000 / 1000))

'093800'

In [12]:
df1 = df.dropna(axis=0, how='any')
len(df1)

29020041

In [14]:
codes = df1['code'].unique()
len(codes)

599

In [16]:
import json
with open('all_codes.txt', 'w') as fp:
    json.dump(list(codes), fp)

In [13]:
df1 = df1.sample(int(0.1 * len(df1)))
len(df1)

2902004

In [ ]:
df1 = df1.set_index('code')

In [39]:
import time
import datetime
t1 = time.time()
time.sleep(10)
t2 = time.time()
t2 -t1

10.000264644622803

In [19]:
t.seconds / 60

0.16666666666666666

In [ ]:
df1 = df1.sort_values(['date','times'], ascending=False)

In [ ]:
train_df = []
test_df = []
pred_df = []
cols_cal_rate = ['open_price','close_price','vwap','volume']

for n, code in tqdm_notebook(enumerate(codes), total=len(codes)):
    # print(n, ':', code)
    group = df1.loc[code]
    # group = group.sort_values(['date','times'], ascending=False)
    g1 = group.set_index(['date', 'times'])
    days = g1.index.get_level_values(0).unique()

    for i in range(7):
        day = days[i]
        df_day = g1.loc[day]
        times = df_day.index.unique()
        for j in range(1, len(times)):
            time = times[j]
            row = df_day.iloc[j, :]
            vwap_next = df_day.iloc[j-1]['vwap']
            label = row['vwap'] / vwap_next

            minute = time / 100000
            feat_hour = [0]*4
            if minute >= 930 and minute < 1030:
                feat_hour[0] = 1
            elif minute >= 1030 and minute <= 1130:
                feat_hour[1] = 1
            elif minute >= 1300 and minute < 1400:
                feat_hour[2] = 1
            elif minute >= 1400 and minute <= 1500:
                feat_hour[3] = 1

            high_price_rate = row['high_price'] / row['open_price']
            low_price_rate = row['low_price'] / row['open_price']
            close_price_rate = row['close_price'] / row['open_price']
            vwap_rate = row['vwap'] / row['open_price']
            volume_log = np.log(row['volume'])
            money_log = np.log(row['money'])

            # 日内前第N分钟与当前的比值特征
            feat_rates_in_day = []
            for k in [1,3,5,10,30,60]:
                rates = [.0] * len(cols_cal_rate)
                if j+1 < len(times):
                    pre_row = df_day.iloc[j+1]
                    for ii, col in enumerate(cols_cal_rate):
                        rates[ii] = pre_row[col] / row[col]
                feat_rates_in_day.extend(rates)

            # 前第N天与当前的比值特征
            feats_rates_days = []
            for k in [1,3,5,10,30,60,90,180]:
                rates = [.0] * len(cols_cal_rate)
                if i+k < len(days):
                    pre_day = days[i+k]
                    # 第N天最后一条成交记录
                    row_pre_day_last = g1.loc[pre_day].iloc[0]
                    for ii, col in enumerate(cols_cal_rate):
                        rates[ii] = row_pre_day_last[col] / row[col]
                feats_rates_days.extend(rates)

            example = [label]
            example.extend(feat_hour)
            example.extend([high_price_rate, low_price_rate, close_price_rate,
                         vwap_rate, volume_log, money_log])
            example.extend(feat_rates_in_day)
            example.extend(feats_rates_days)
            if i == 0:
                pred_df.append(example)
            if i == 1:
                test_df.append(example)
            else:
                train_df.append(example)
len(train_df), len(test_df), len(pred_df)

<ipython-input-29-713dc423bcfd>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n, code in tqdm_notebook(enumerate(codes), total=len(codes)):


  0%|          | 0/599 [00:00<?, ?it/s]

(82199, 13719, 13657)

In [ ]:
train_df = np.array(train_df)
test_df = np.array(test_df)
val_df = np.array(pred_df)

In [ ]:
# 训练集打散
np.random.shuffle(train_df)
# 测试集、验证集随机重抽样
rand_index = np.random.choice(len(test_df), len(test_df), replace=True)
test_df = test_df[rand_index, :]
rand_index = np.random.choice(len(val_df), len(val_df), replace=True)
val_df = np.array(val_df)[rand_index, :]
len(train_df), len(test_df), len(val_df)

(82199, 13719, 13657)

In [ ]:
# 有inf值，先截断处理
train_df1 = np.clip(train_df, 0, 1000)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
train_x = train_df1[:, 1:]
train_y = train_df1[:, 0]
test_x = test_df[:, 1:]
test_y = test_df[:, 0]
val_x = val_df[:, 1:]
val_y = val_df[:, 0]

model = LinearRegression()
model.fit(train_x, train_y)
model.coef_[:5]

array([-9.06967718e-05, -1.79143577e-04, -1.00788576e-06,  2.70848233e-04,
       -1.22585859e-01])

In [ ]:
def mean_mse(y_pred, y_true):
    assert len(y_pred) > 0
    assert len(y_pred) == len(y_true)
    y = np.sum(np.square(np.subtract(y_pred, y_true)))
    result = y / len(y_pred)
    return round(result, 8)
# expect(5)
mean_mse([1,2], [4, 1])

In [ ]:
def mean_mae(y_pred, y_true):
    assert len(y_pred) > 0
    assert len(y_pred) == len(y_true)
    y = np.sum(np.abs(np.subtract(y_pred, y_true)))
    result = y / len(y_pred)
    return round(result, 8)
# expect(2)
mean_mae([1,2], [4, 1])

In [ ]:
def mean_mape(y_pred, y_true):
    assert len(y_pred) > 0
    assert len(y_pred) == len(y_true)
    x = np.divide(np.abs(np.subtract(y_pred, y_true)), y_true)
    y = np.sum(x)
    result = y / len(y_pred)
    return round(result, 8)
# expect((0.75+1)/2=0.875)
mean_mape([1,2], [4, 1])

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [ ]:
y = model.predict(test_x)
mean_absolute_error(y, test_y), mean_squared_error(y, test_y),mean_absolute_percentage_error(y, test_y)

(0.0020107589517638033, 1.0928917612984247e-05, 0.002010727449174488)

In [ ]:
y = model.predict(val_x)
mean_absolute_error(y, val_y), mean_squared_error(y, val_y),mean_absolute_percentage_error(y, val_y)

(0.0019177737866514506, 9.624656305924681e-06, 0.0019176354101346478)

In [ ]:
save_path = os.path.join('/workspace/user_code/quant_campaign', 'lr.pk')
print("=====> save")
joblib.dump(model, save_path)

=====> save


['/workspace/user_code/quant_campaign/lr.pk']